In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab
from collections import Counter
import pickle
import itertools

Choose the dataset

In [4]:
#dataset = 'hospital'
#dataset = 'high_school'
#dataset = 'conference'
dataset = 'workplace'
#dataset = 'pr_school'

filename is the path to the datsset, like "data/primaryschool.csv"

In [58]:
ls Datasets

conf.dat  cprepa.csv  hospital.dat  office.dat  school.csv


In [88]:
filename = './Datasets/school.csv'

In [89]:
person1_list = []
person2_list = []
for line in open(filename):
    csv_row = line.split()
    person1_list.append(int(csv_row[1]))
    person2_list.append(int(csv_row[2]))

Reassign nodes identities such that they start from 0 and go up to N:

In [90]:
nodes = np.unique(person1_list+person2_list)
nodes

array([1426, 1427, 1428, 1429, 1430, 1431, 1434, 1435, 1437, 1439, 1441,
       1443, 1451, 1452, 1453, 1457, 1458, 1459, 1461, 1465, 1468, 1471,
       1475, 1477, 1479, 1480, 1482, 1483, 1486, 1489, 1493, 1495, 1498,
       1500, 1501, 1502, 1503, 1504, 1511, 1516, 1519, 1520, 1521, 1522,
       1524, 1525, 1528, 1532, 1533, 1538, 1539, 1545, 1546, 1548, 1549,
       1551, 1552, 1555, 1558, 1560, 1562, 1563, 1564, 1567, 1570, 1572,
       1574, 1578, 1579, 1580, 1585, 1592, 1594, 1601, 1603, 1604, 1606,
       1609, 1613, 1616, 1617, 1618, 1625, 1628, 1630, 1632, 1637, 1641,
       1643, 1647, 1648, 1649, 1650, 1653, 1656, 1661, 1663, 1664, 1665,
       1666, 1668, 1670, 1673, 1674, 1675, 1680, 1681, 1682, 1684, 1685,
       1687, 1688, 1695, 1696, 1697, 1698, 1700, 1702, 1704, 1705, 1706,
       1707, 1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1718, 1719,
       1720, 1722, 1723, 1727, 1730, 1731, 1732, 1735, 1737, 1738, 1739,
       1741, 1743, 1744, 1745, 1746, 1748, 1749, 17

In [91]:
new_nodes = np.arange(len(nodes))
new_nodes

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [92]:
conversion = {}
for i in range(len(nodes)):
    conversion[nodes[i]] = new_nodes[i]

In [93]:
new_person1_list = []
for node in person1_list:
    new_person1_list.append(conversion[node])
    
new_person2_list = []
for node in person2_list:
    new_person2_list.append(conversion[node])

In [94]:
Dpairwise = {}
for n in range(len(new_person1_list)):
    edge = [new_person1_list[n],new_person2_list[n]]
    if tuple(edge) in Dpairwise:
        Dpairwise[tuple(edge)] += 1
    else:
        Dpairwise[tuple(edge)] = 1 

Normalize pairwise:

In [95]:
print(max(Dpairwise.values()))

764


In [96]:
meta

,new index,class,strength
0,0,x,0.000768
1,1,x,0.347158
2,2,x,0.291091
3,3,x,0.541475
4,4,x,0.877880
...,...,...,...
212,212,x,0.366359
213,213,x,1.020737
214,214,x,0.365591
215,215,x,1.040707


In [16]:
Dpairwise_n = {}
for key in Dpairwise:
    Dpairwise_n[key] = Dpairwise[key]/max(Dpairwise.values())

# Format for simple contagion

In [35]:
import pandas as pd
import json
import gzip

In [51]:
network_data = {'type':'daily'}
G = [(int(u),int(v),float(w) )  for (u,v),w in Dpairwise_n.items()]
network_data['graphs'] = [ G ]
def get_strength(G,u):
    s = 0
    for x,y,w in G:
        if u==x or u==y:
            s += w
    return s        
meta =  pd.DataFrame( [(u, 'x', get_strength(G,u)) for u in new_nodes], columns= ('new index', 'class', 'strength') )
network_data['metadata'] = meta.to_json()

In [52]:
js = json.dumps( network_data , separators=(',', ':') )
with gzip.open('office.json.gzip', 'w') as fout:
    json_bytes = js.encode('utf-8')
    fout.write(json_bytes)

Save all pairwise:

In [18]:
dict_name = 'pairwise/%s_strength_edgelist_pairwise_sorted_norm.pkl'%dataset
with open(dict_name, 'wb') as f:
    pickle.dump(Dpairwise_n, f)